In [1]:
!pip3 install optuna==0.14.0
!pip3 install optkeras==0.0.7
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 91 kB 4.0 MB/s 
     |████████████████████████████████| 81 kB 8.8 MB/s 
     |████████████████████████████████| 209 kB 44.2 MB/s 
     |████████████████████████████████| 78 kB 7.3 MB/s 
     |████████████████████████████████| 78 kB 5.8 MB/s 
     |████████████████████████████████| 112 kB 83.6 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 147 kB 74.7 MB/s 
  Created wheel for optuna: filename=optuna-0.14.0-py3-none-any.whl size=125709 sha256=ab4ef902244a10dd0084674f07e692659f81404efdb173f3a01c17e27753a793
  Stored in directory: /root/.cache/pip/wheels/c1/41/64/03b183676c5d5e978de160cab6268d5b4fb095dff63f720e01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=61565d86d76ebe0573f40f91abeb88c0af81f95b20a955e7fff0f373fd1a22da
  Stored in directory: /root/.ca

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 70.2 MB/s 


In [2]:
import numpy as np
import pandas as pd
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Activation, Flatten, Dense, Conv2D, Conv1D,Input
from keras.layers import MaxPooling1D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, Adagrad, RMSprop, Adam, Adadelta, Adamax, Nadam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from keras.layers.core import *
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import keras.backend as K
import optuna
import math

In [3]:
import keras
print('Keras', keras.__version__)

import tensorflow as tf
print('TensorFlow', tf.__version__)

# import Optuna and OptKeras after Keras
import optuna 
print('Optuna', optuna.__version__)

from optkeras.optkeras import OptKeras
import optkeras
print('OptKeras', optkeras.__version__)

Keras 2.8.0
TensorFlow 2.8.2
Optuna 0.14.0
OptKeras 0.0.7


In [4]:
train_df = pd.read_parquet("/content/drive/MyDrive/New Data/18features/18_train_main.parquet.snappy",engine='fastparquet')
test_df = pd.read_parquet("/content/drive/MyDrive/New Data/18features/18_test_main.parquet.snappy",engine='fastparquet')

In [5]:
# prepare the cross-validation procedure
kfold = KFold(n_splits=3, shuffle=True)

training_withaim=train_df.drop(labels="r_id", axis=1)
testing_withaim=test_df.drop(labels="r_id", axis=1)

#Check the NaN in data and drop them
imp_train=SimpleImputer(missing_values=np.NaN)
training=pd.DataFrame(imp_train.fit_transform(training_withaim))

imp_test=SimpleImputer(missing_values=np.NaN)
testing=pd.DataFrame(imp_test.fit_transform(testing_withaim))
print(training)
# There aren't any nan data in the dataframe 
training.isnull().values.sum()
testing.isnull().values.sum()

         0     1      2      3       4           5         6    7     8   \
0       5.0  25.0   80.0  173.0   129.0  108.203988  0.178295  3.0   6.0   
1       4.0  16.0  171.0  128.0   330.0   58.652416  0.909091  7.0  21.0   
2       3.0   9.0  114.0  277.0  1064.0   64.378573  1.753759  2.0  57.0   
3       5.0  25.0   34.0   35.0    59.0  102.077493  0.067797  0.0   0.0   
4       1.0   1.0  261.0  137.0     5.0   44.994772  0.000000  0.0   0.0   
...     ...   ...    ...    ...     ...         ...       ...  ...   ...   
399995  5.0  25.0  191.0    1.0    16.0    0.000110  0.000000  0.0   0.0   
399996  3.0   9.0  434.0  164.0   471.0   70.677502  0.116773  4.0  22.0   
399997  5.0  25.0  131.0  211.0     7.0   23.186602  0.142857  0.0   1.0   
399998  5.0  25.0   49.0    1.0    16.0    6.598250  0.062500  0.0   0.0   
399999  1.0   1.0   25.0    1.0    58.0   27.243015  0.034483  0.0   0.0   

          9       10   11      12   13     14        15        16     17    18  
0     

0

In [6]:
training = training.iloc[: , 0:18]
testing = testing.iloc[: , 0:18]
training

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,5.0,25.0,80.0,173.0,129.0,108.203988,0.178295,3.0,6.0,4.50,118.0,5.0,10.0,6.0,60.0,0.279954,0.433241,66.44
1,4.0,16.0,171.0,128.0,330.0,58.652416,0.909091,7.0,21.0,3.84,891.0,4.0,711.0,6.0,48.0,0.272727,0.474242,72.66
2,3.0,9.0,114.0,277.0,1064.0,64.378573,1.753759,2.0,57.0,4.21,9948.0,3.5,154.0,6.0,33.0,0.146667,0.294583,68.13
3,5.0,25.0,34.0,35.0,59.0,102.077493,0.067797,0.0,0.0,4.30,48.0,4.0,109.0,7.0,71.0,0.383333,0.541667,76.93
4,1.0,1.0,261.0,137.0,5.0,44.994772,0.000000,0.0,0.0,3.40,5.0,4.0,18.0,5.0,42.5,0.039118,0.300063,88.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,5.0,25.0,191.0,1.0,16.0,0.000110,0.000000,0.0,0.0,4.72,9.0,4.0,30.0,6.0,60.0,-0.130221,0.420113,69.11
399996,3.0,9.0,434.0,164.0,471.0,70.677502,0.116773,4.0,22.0,3.62,716.0,4.5,454.0,6.0,64.0,0.241433,0.523121,77.16
399997,5.0,25.0,131.0,211.0,7.0,23.186602,0.142857,0.0,1.0,2.93,3.0,4.0,380.0,7.0,72.0,0.084722,0.475076,82.24
399998,5.0,25.0,49.0,1.0,16.0,6.598250,0.062500,0.0,0.0,5.00,3.0,4.0,1984.0,7.0,60.0,0.340000,0.460000,78.45


In [7]:
scaler = StandardScaler()

# scale skewed and target features
std_train_df = train_df.copy(deep=True)
std_train_df = scaler.fit_transform(training)
std_test_df = test_df.copy(deep=True)
std_test_df = scaler.transform(testing)
#std_test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']] = scaler.transform(test_df[['r_stars','r_stars_square','r_length', 'u_friends_count', 'u_review_count', 'u_month_age', 'b_stars','b_review_count','r_rea']])

print(std_train_df[0])
std_train_df = pd.DataFrame(std_train_df)
std_test_df = pd.DataFrame(std_test_df)
std_train_df

[ 0.90216754  0.99645674 -0.4633628  -0.02815708 -0.12709163  1.889782
 -0.11838612  0.3622633  -0.14541076  0.94854255 -0.15259145  1.58161918
 -0.48965899 -0.58778166 -0.41108489  0.50629158 -0.08015604 -1.15116974]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.902168,0.996457,-0.463363,-0.028157,-0.127092,1.889782,-0.118386,0.362263,-0.145411,0.948543,-0.152591,1.581619,-0.489659,-0.587782,-0.411085,0.506292,-0.080156,-1.151170
1,0.252164,0.060934,0.330297,-0.108118,0.288456,0.458212,0.116132,1.662180,-0.029172,0.144179,0.016366,0.321906,0.559060,-0.587782,-0.797300,0.468778,0.199067,-0.623535
2,-0.397840,-0.666695,-0.166830,0.156643,1.805931,0.623643,0.387193,0.037284,0.249799,0.595110,1.995984,-0.307951,-0.274230,-0.587782,-1.280069,-0.185575,-1.024437,-1.007809
3,0.902168,0.996457,-0.864554,-0.273372,-0.271810,1.712784,-0.153846,-0.612674,-0.191906,0.704796,-0.167892,0.321906,-0.341552,0.617819,-0.057054,1.042912,0.658240,-0.261316
4,-1.697847,-1.498271,1.115236,-0.092126,-0.383450,0.063635,-0.175602,-0.612674,-0.191906,-0.392064,-0.177290,0.321906,-0.477691,-1.793382,-0.974315,-0.743841,-0.987117,0.760023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,0.902168,0.996457,0.504728,-0.333787,-0.360708,-1.236284,-0.175602,-0.612674,-0.191906,1.216664,-0.176416,0.321906,-0.459738,-0.587782,-0.411085,-1.622847,-0.169560,-0.924677
399996,-0.397840,-0.666695,2.624063,-0.044149,0.579960,0.805623,-0.138129,0.687243,-0.021423,-0.123943,-0.021885,0.951762,0.174580,-0.587782,-0.282347,0.306337,0.531939,-0.241805
399997,0.902168,0.996457,-0.018564,0.039366,-0.379315,-0.566414,-0.129758,-0.612674,-0.184157,-0.964869,-0.177727,0.321906,0.063873,0.617819,-0.024870,-0.507120,0.204746,0.189125
399998,0.902168,0.996457,-0.733731,-0.333787,-0.360708,-1.045660,-0.155546,-0.612674,-0.191906,1.557909,-0.177727,0.321906,2.463511,0.617819,-0.411085,0.817979,0.102077,-0.132376


In [8]:
training = std_train_df.iloc[: , 0:18]
testing = std_test_df.iloc[: , 0:18]
labelsForTrain=training_withaim.iloc[: , -1]
labelsForTest=testing_withaim.iloc[: , -1]
input_shape = training.shape

In [9]:
training.shape

(400000, 18)

In [10]:
def create_model(activation, num_hidden_layer, num_hidden_unit):
  inputs = Input(shape=(training.shape[1],))
  model = inputs
  for i in range(1,num_hidden_layer):
    model = Dense(num_hidden_unit, activation=activation,)(model)
        
        
  model = Dense(1,)(model)
  model = Model(inputs, model)

  return model

In [11]:
def create_model_cnn(activation, num_hidden_layer, num_hidden_unit, kernel_size, filter):
  model_cnn = Sequential()
  model_cnn.add(Conv1D(filter, kernel_size, activation=activation, input_shape=(18,1)))
  model_cnn.add(MaxPooling1D())
  model_cnn.add(Flatten())
  for i in range(1,num_hidden_layer):
    model_cnn.add(Dense(num_hidden_unit, activation=activation,))

  model_cnn.add(Dense(1))

  return model_cnn

In [12]:
def objective_cnn(trial):
  K.clear_session()
    
  activation = trial.suggest_categorical('activation',['relu','tanh','linear'])
  optimizer = trial.suggest_categorical('optimizer',['adam','rmsprop','adagrad', 'sgd'])
    
  num_hidden_layer = trial.suggest_int('num_hidden_layer',1,5)
  num_hidden_unit = trial.suggest_int('num_hidden_unit',10,100)
  kernel_size = trial.suggest_int('kernel_size',1,5)
  filter = trial.suggest_int('filter',1,100)
    
  learning_rate = trial.suggest_loguniform('learning_rate', 0.00001,0.1)
  if optimizer == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer == 'adagrad':
    optimizer = Adagrad(learning_rate=learning_rate)
  elif optimizer =='rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  elif optimizer =='sgd':
    optimizer = SGD(learning_rate=learning_rate)
    
  num_folds = 3
  kfold=KFold(n_splits=3,shuffle=True)
  fold_no=1
  loss_per_fold = []
  es = EarlyStopping(monitor='mse', patience=50)
  model = create_model_cnn(activation, num_hidden_layer, num_hidden_unit, kernel_size, filter)
  model_list_cnn.append(model)
  model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
  for train,test in kfold.split(training,labelsForTrain):
    scores=model.evaluate(testing,labelsForTest,verbose=0)

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
  
    # Fit data to model
    history = model.fit(training, labelsForTrain,
                batch_size=20,
                epochs=10,
                verbose=2,
                callbacks=[es])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
    loss_per_fold.append(scores[0])
    # Increase fold number
    fold_no = fold_no + 1  
  
  history_list_cnn.append(history)
    
  mse = np.array(history.history['mse'])
    
  return mse[-1]

In [13]:
def objective(trial):
  K.clear_session()
    
  activation = trial.suggest_categorical('activation',['relu','tanh','linear'])
  optimizer = trial.suggest_categorical('optimizer',['adam','rmsprop','adagrad', 'sgd'])
    
  num_hidden_layer = trial.suggest_int('num_hidden_layer',1,5)
  #get more features per layer, add num of hidden unit if have time
  num_hidden_unit = trial.suggest_int('num_hidden_unit',10,100)
    
  learning_rate = trial.suggest_loguniform('learning_rate', 0.00001,0.1)
  if optimizer == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer == 'adagrad':
    optimizer = Adagrad(learning_rate=learning_rate)
  elif optimizer =='rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  elif optimizer =='sgd':
    optimizer = SGD(learning_rate=learning_rate)
    
  num_folds = 3
  kfold=KFold(n_splits=3,shuffle=True)
  fold_no=1
  loss_per_fold = []
  es = EarlyStopping(monitor='mse', patience=50)
  model = create_model(activation, num_hidden_layer, num_hidden_unit)
  model_list.append(model)
  model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
  for train,test in kfold.split(training,labelsForTrain):
    scores=model.evaluate(testing,labelsForTest,verbose=0)

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
  
    # Fit data to model
    history = model.fit(training, labelsForTrain,
                batch_size=20,
                epochs=10,
                verbose=2,
                callbacks=[es])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
    loss_per_fold.append(scores[0])
    # Increase fold number
    fold_no = fold_no + 1  
  
  history_list.append(history)
    
  mse = np.array(history.history['mse'])
    
  return mse[-1]

In [ ]:
model_list_cnn=[]
history_list_cnn=[]
study_name = 'CNN_study'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
study.optimize(objective_cnn, n_trials=50, )

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 39s - loss: 18.1846 - mse: 18.1846 - mae: 1.6748 - 39s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 39s - loss: 18.1968 - mse: 18.1968 - mae: 1.6916 - 39s/epoch - 2ms/step
Epoch 3/10
20000/20000 - 41s - loss: 17.9370 - mse: 17.9370 - mae: 1.6810 - 41s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 38s - loss: 17.7152 - mse: 17.7152 - mae: 1.6856 - 38s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 38s - loss: 17.9484 - mse: 17.9484 - mae: 1.6666 - 38s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 38s - loss: 17.6137 - mse: 17.6137 - mae: 1.6608 - 38s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 37s - loss: 17.6848 - mse: 17.6848 - mae: 1.6780 - 37s/epoch - 2ms/step
Epoch 8/10
20000/20000 - 37s - loss: 17.8561 - mse: 17.8561 - mae: 1.6794 - 37s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 36s - loss: 18.0299 - mse: 18.0299 - mae: 1.7310 - 36s/epoch - 2ms/step
Epoch 10/10
20000/20000 - 37s 

[I 2022-09-30 03:23:48,408] Finished trial#0 resulted in value: 19.510541915893555. Current best value is 19.510541915893555 with parameters: {'activation': 'tanh', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 35, 'kernel_size': 5, 'filter': 60, 'learning_rate': 0.004538346443656373}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 29s - loss: 18.1093 - mse: 18.1093 - mae: 1.6580 - 29s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 28s - loss: 17.7154 - mse: 17.7154 - mae: 1.6366 - 28s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 29s - loss: 17.5829 - mse: 17.5829 - mae: 1.6334 - 29s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 29s - loss: 17.8670 - mse: 17.8670 - mae: 1.6332 - 29s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 28s - loss: 17.5546 - mse: 17.5546 - mae: 1.6355 - 28s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 29s - loss: 17.6939 - mse: 17.6939 - mae: 1.6335 - 29s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 29s - loss: 17.4680 - mse: 17.4680 - mae: 1.6288 - 29s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 28s - loss: 17.9172 - mse: 17.9172 - mae: 1.6347 - 28s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 28s - loss: 17.5001 - mse: 17.5001 - mae: 1.6333 - 28s/epoch - 1ms/step
Epoch 10/10
20000/20000 - 28s 

[I 2022-09-30 03:37:56,692] Finished trial#1 resulted in value: 17.741100311279297. Current best value is 17.741100311279297 with parameters: {'activation': 'linear', 'optimizer': 'rmsprop', 'num_hidden_layer': 5, 'num_hidden_unit': 96, 'kernel_size': 2, 'filter': 14, 'learning_rate': 0.004102079349565866}.


Score for fold 3: loss of 12.804410934448242
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 29s - loss: 19.9669 - mse: 19.9669 - mae: 1.7517 - 29s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 28s - loss: 18.9033 - mse: 18.9033 - mae: 1.6926 - 28s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 29s - loss: 18.6682 - mse: 18.6682 - mae: 1.6692 - 29s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 29s - loss: 18.5256 - mse: 18.5256 - mae: 1.6599 - 29s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 30s - loss: 18.4173 - mse: 18.4173 - mae: 1.6531 - 30s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 30s - loss: 18.3251 - mse: 18.3251 - mae: 1.6463 - 30s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 30s - loss: 18.2454 - mse: 18.2454 - mae: 1.6404 - 30s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 30s - loss: 18.1761 - mse: 18.1761 - mae: 1.6351 - 30s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 31s - loss: 18.1150 - mse: 18.1150 - mae: 1.6352 - 31s/ep

...

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 25s - loss: 17.1156 - mse: 17.1156 - mae: 1.6059 - 25s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 24s - loss: 16.1366 - mse: 16.1366 - mae: 1.5490 - 24s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 24s - loss: 15.9060 - mse: 15.9060 - mae: 1.5370 - 24s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 24s - loss: 15.7938 - mse: 15.7938 - mae: 1.5371 - 24s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 24s - loss: 15.6450 - mse: 15.6450 - mae: 1.5292 - 24s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 24s - loss: 15.6468 - mse: 15.6468 - mae: 1.5248 - 24s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 23s - loss: 15.7050 - mse: 15.7050 - mae: 1.5294 - 23s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 23s - loss: 15.6968 - mse: 15.6968 - mae: 1.5313 - 23s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 25s - loss: 15.5835 - mse: 15.5835 - mae: 1.5255 - 25s/epoch - 1ms/step
Epoch 10/10
20000/20000 - 24s 

[I 2022-09-30 14:22:44,173] Finished trial#47 resulted in value: 14.892133712768555. Current best value is 14.38178539276123 with parameters: {'activation': 'tanh', 'optimizer': 'sgd', 'num_hidden_layer': 2, 'num_hidden_unit': 81, 'kernel_size': 2, 'filter': 40, 'learning_rate': 0.000693576191437307}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 26s - loss: 17.5871 - mse: 17.5871 - mae: 1.6167 - 26s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 26s - loss: 16.5663 - mse: 16.5663 - mae: 1.5715 - 26s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 26s - loss: 16.1452 - mse: 16.1452 - mae: 1.5472 - 26s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 26s - loss: 15.8713 - mse: 15.8713 - mae: 1.5282 - 26s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 26s - loss: 15.7001 - mse: 15.7001 - mae: 1.5195 - 26s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 27s - loss: 15.5796 - mse: 15.5796 - mae: 1.5129 - 27s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 26s - loss: 15.4305 - mse: 15.4305 - mae: 1.5063 - 26s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 26s - loss: 15.3667 - mse: 15.3667 - mae: 1.5023 - 26s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 27s - loss: 15.3119 - mse: 15.3119 - mae: 1.5006 - 27s/epoch - 1ms/step
Epoch 10/10
20000/20000 - 27s 

[I 2022-09-30 14:43:55,882] Finished trial#48 resulted in value: 14.484635353088379. Current best value is 14.38178539276123 with parameters: {'activation': 'tanh', 'optimizer': 'sgd', 'num_hidden_layer': 2, 'num_hidden_unit': 81, 'kernel_size': 2, 'filter': 40, 'learning_rate': 0.000693576191437307}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 27s - loss: 18.0008 - mse: 18.0008 - mae: 1.6323 - 27s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 27s - loss: 16.9701 - mse: 16.9701 - mae: 1.5841 - 27s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 28s - loss: 16.5931 - mse: 16.5931 - mae: 1.5699 - 28s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 27s - loss: 16.2968 - mse: 16.2968 - mae: 1.5553 - 27s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 26s - loss: 16.0707 - mse: 16.0707 - mae: 1.5410 - 26s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 27s - loss: 15.8990 - mse: 15.8990 - mae: 1.5293 - 27s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 26s - loss: 15.7580 - mse: 15.7580 - mae: 1.5211 - 26s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 31s - loss: 15.6356 - mse: 15.6356 - mae: 1.5131 - 31s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 32s - loss: 15.5570 - mse: 15.5570 - mae: 1.5076 - 32s/epoch - 2ms/step
Epoch 10/10
20000/20000 - 31s 

In [15]:
#{'activation': 'tanh', 'optimizer': 'sgd', 'num_hidden_layer': 2, 'num_hidden_unit': 81, 'kernel_size': 2, 'filter': 40, 'learning_rate': 0.000693576191437307}
optimizer = SGD(learning_rate=0.000693576191437307)

num_folds = 3
kfold=KFold(n_splits=3,shuffle=True)
fold_no=1
loss_per_fold = []
model = create_model_cnn(activation='tanh', num_hidden_layer=2, num_hidden_unit=81, kernel_size=2, filter=40)
model.summary()
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
model.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
for train,test in kfold.split(training,labelsForTrain):
  scores=model.evaluate(testing,labelsForTest,verbose=0)

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  history = model.fit(training, labelsForTrain,
                batch_size=20,
                epochs=20,
                verbose=2)
    
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1    

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 17, 40)            120       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 8, 40)            0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 320)               0         
                                                                 
 dense_2 (Dense)             (None, 81)                26001     
                                                                 
 dense_3 (Dense)             (None, 1)                 82        
                                                                 
Total params: 26,203
Trainable params: 26,203
Non-trainable params: 0
__________________________________________________

In [ ]:
#Mean mse for train is 14.289 (3.780) best is 13.844(3.72), mean mae for train is 1.469
results = model_cnn_best.evaluate(testing, labelsForTest, batch_size=20)

5000/5000 [==============================] - 8s 2ms/step - loss: 13.6385 - mse: 13.6385 - mae: 2.0673


In [ ]:
model_list=[]
history_list=[]
study_name = 'NN_study'
study = optuna.create_study(study_name=study_name, load_if_exists=True)
#15.1
study.optimize(objective, n_trials=50, )

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 24s - loss: 21.3438 - mse: 21.3438 - mae: 1.6281 - 24s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 24s - loss: 17.2346 - mse: 17.2346 - mae: 1.6124 - 24s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 22s - loss: 17.0615 - mse: 17.0615 - mae: 1.6092 - 22s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 22s - loss: 17.1603 - mse: 17.1603 - mae: 1.6069 - 22s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 21s - loss: 17.0885 - mse: 17.0885 - mae: 1.6054 - 21s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 22s - loss: 17.0414 - mse: 17.0414 - mae: 1.6084 - 22s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 21s - loss: 17.0386 - mse: 17.0386 - mae: 1.6001 - 21s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 21s - loss: 17.0484 - mse: 17.0484 - mae: 1.6071 - 21s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 21s - loss: 17.1014 - mse: 17.1014 - mae: 1.6059 - 21s/epoch - 1ms/step
Epoch 10/10
20000/20000 - 21s 

[I 2022-09-30 23:28:03,962] Finished trial#0 resulted in value: 16.997882843017578. Current best value is 16.997882843017578 with parameters: {'activation': 'linear', 'optimizer': 'adagrad', 'num_hidden_layer': 5, 'num_hidden_unit': 54, 'lear  ning_rate': 0.07760938712352267}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 26s - loss: 17.7250 - mse: 17.7250 - mae: 1.5966 - 26s/epoch - 1ms/step
Epoch 2/10
20000/20000 - 25s - loss: 17.4547 - mse: 17.4547 - mae: 1.5840 - 25s/epoch - 1ms/step
Epoch 3/10
20000/20000 - 26s - loss: 17.4883 - mse: 17.4883 - mae: 1.5842 - 26s/epoch - 1ms/step
Epoch 4/10
20000/20000 - 25s - loss: 17.4228 - mse: 17.4228 - mae: 1.5821 - 25s/epoch - 1ms/step
Epoch 5/10
20000/20000 - 25s - loss: 17.5935 - mse: 17.5935 - mae: 1.5837 - 25s/epoch - 1ms/step
Epoch 6/10
20000/20000 - 24s - loss: 17.4588 - mse: 17.4588 - mae: 1.5835 - 24s/epoch - 1ms/step
Epoch 7/10
20000/20000 - 25s - loss: 17.6610 - mse: 17.6610 - mae: 1.5848 - 25s/epoch - 1ms/step
Epoch 8/10
20000/20000 - 25s - loss: 17.4965 - mse: 17.4964 - mae: 1.5816 - 25s/epoch - 1ms/step
Epoch 9/10
20000/20000 - 25s - loss: 17.5655 - mse: 17.5655 - mae: 1.5826 - 25s/epoch - 1ms/step
Epoch 10/10
20000/20000 - 25s 

[I 2022-09-30 23:40:46,662] Finished trial#1 resulted in value: 17.527057647705078. Current best value is 16.997882843017578 with parameters: {'activation': 'linear', 'optimizer': 'adagrad', 'num_hidden_layer': 5, 'num_hidden_unit': 54, 'lear  ning_rate': 0.07760938712352267}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 20s - loss: 17.3850 - mse: 17.3850 - mae: 1.6221 - 20s/epoch - 990us/step
Epoch 2/10
20000/20000 - 19s - loss: 17.1986 - mse: 17.1986 - mae: 1.6197 - 19s/epoch - 968us/step
Epoch 3/10
20000/20000 - 20s - loss: 17.2591 - mse: 17.2591 - mae: 1.6204 - 20s/epoch - 991us/step
Epoch 4/10
20000/20000 - 19s - loss: 17.1495 - mse: 17.1495 - mae: 1.6179 - 19s/epoch - 950us/step
Epoch 5/10
20000/20000 - 20s - loss: 17.2746 - mse: 17.2746 - mae: 1.6223 - 20s/epoch - 989us/step
Epoch 6/10
20000/20000 - 20s - loss: 17.2683 - mse: 17.2683 - mae: 1.6185 - 20s/epoch - 993us/step
Epoch 7/10
20000/20000 - 20s - loss: 17.1730 - mse: 17.1730 - mae: 1.6179 - 20s/epoch - 980us/step
Epoch 8/10
20000/20000 - 19s - loss: 17.2308 - mse: 17.2308 - mae: 1.6219 - 19s/epoch - 962us/step
Epoch 9/10
20000/20000 - 20s - loss: 17.2804 - mse: 17.2804 - mae: 1.6189 - 20s/epoch - 995us/step
Epoch 10/10


...

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 34s - loss: 17.4333 - mse: 17.4333 - mae: 1.6630 - 34s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 32s - loss: 17.0160 - mse: 17.0160 - mae: 1.6652 - 32s/epoch - 2ms/step
Epoch 3/10
20000/20000 - 33s - loss: 16.9377 - mse: 16.9377 - mae: 1.6438 - 33s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 33s - loss: 17.1310 - mse: 17.1310 - mae: 1.6688 - 33s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 34s - loss: 17.1850 - mse: 17.1850 - mae: 1.6525 - 34s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 33s - loss: 17.0258 - mse: 17.0258 - mae: 1.6541 - 33s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 31s - loss: 16.9604 - mse: 16.9604 - mae: 1.6501 - 31s/epoch - 2ms/step
Epoch 8/10
20000/20000 - 31s - loss: 16.9535 - mse: 16.9535 - mae: 1.6474 - 31s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 32s - loss: 16.9897 - mse: 16.9897 - mae: 1.6609 - 32s/epoch - 2ms/step
Epoch 10/10
20000/20000 - 31s 

[I 2022-10-01 09:17:01,185] Finished trial#39 resulted in value: 17.163469314575195. Current best value is 10.764413833618164 with parameters: {'activation': 'relu', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 60, 'lear  ning_rate': 0.0003135813689247155}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 39s - loss: 18.6139 - mse: 18.6139 - mae: 1.6729 - 39s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 38s - loss: 18.2722 - mse: 18.2722 - mae: 1.6676 - 38s/epoch - 2ms/step
Epoch 3/10
20000/20000 - 38s - loss: 17.7451 - mse: 17.7451 - mae: 1.6499 - 38s/epoch - 2ms/step
Epoch 4/10
20000/20000 - 38s - loss: 17.7081 - mse: 17.7081 - mae: 1.6522 - 38s/epoch - 2ms/step
Epoch 5/10
20000/20000 - 37s - loss: 17.7031 - mse: 17.7031 - mae: 1.6456 - 37s/epoch - 2ms/step
Epoch 6/10
20000/20000 - 38s - loss: 17.7266 - mse: 17.7266 - mae: 1.6420 - 38s/epoch - 2ms/step
Epoch 7/10
20000/20000 - 38s - loss: 17.5827 - mse: 17.5827 - mae: 1.6454 - 38s/epoch - 2ms/step
Epoch 8/10
20000/20000 - 39s - loss: 17.5714 - mse: 17.5714 - mae: 1.6301 - 39s/epoch - 2ms/step
Epoch 9/10
20000/20000 - 40s - loss: 17.7418 - mse: 17.7418 - mae: 1.6299 - 40s/epoch - 2ms/step
Epoch 10/10
20000/20000 - 38s 

[I 2022-10-01 09:36:36,951] Finished trial#40 resulted in value: 17.44825553894043. Current best value is 10.764413833618164 with parameters: {'activation': 'relu', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 60, 'lear  ning_rate': 0.0003135813689247155}.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
20000/20000 - 40s - loss: 16.8133 - mse: 16.8133 - mae: 1.5636 - 40s/epoch - 2ms/step
Epoch 2/10
20000/20000 - 40s - loss: 15.9406 - mse: 15.9406 - mae: 1.5195 - 40s/epoch - 2ms/step
Epoch 3/10


In [15]:
num_folds = 3

optimizer = Adam(learning_rate=0.0003135813689247155)

kfold=KFold(n_splits=3,shuffle=True)
fold_no=1
loss_per_fold = []
#{'activation': 'relu', 'optimizer': 'adam', 'num_hidden_layer': 5, 'num_hidden_unit': 60, 'learning_rate': 0.0003135813689247155}
model_nn_best = create_model(activation='relu', num_hidden_layer=5, num_hidden_unit=60)
model_nn_best.summary()
model_nn_best.compile(loss='mse',optimizer=optimizer,metrics=['mse','mae'])
for train,test in kfold.split(training,labelsForTrain):
  scores=model_nn_best.evaluate(testing,labelsForTest,verbose=0)

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')
  
  # Fit data to model
  history = model_nn_best.fit(training, labelsForTrain,
              batch_size=20,
              #The result from epoch 20 and epoch 200 don't make too much differenct
              epochs=100,
              verbose=2,)
    
  print(f'Score for fold {fold_no}: {model_nn_best.metrics_names[0]} of {scores[0]}')
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1  

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 18)]              0         
                                                                 
 dense (Dense)               (None, 60)                1140      
                                                                 
 dense_1 (Dense)             (None, 60)                3660      
                                                                 
 dense_2 (Dense)             (None, 60)                3660      
                                                                 
 dense_3 (Dense)             (None, 60)                3660      
                                                                 
 dense_4 (Dense)             (None, 1)                 61        
                                                                 
Total params: 12,181
Trainable params: 12,181
Non-trainable p

In [17]:
results = model_nn_best.evaluate(testing, labelsForTest, batch_size=10)

10000/10000 [==============================] - 13s 1ms/step - loss: 10.6730 - mse: 10.6730 - mae: 1.4169
